In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/BCI')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/BCI


In [2]:
!pip install mne

     |████████████████████████████████| 6.8MB 3.5MB/s 


In [3]:
import numpy as np
import scipy.linalg as la
import mne.filter as bandpass
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from scipy.io import loadmat  

In [4]:
class MI:
    def __init__(self):
        self.V=None
      
    def bandpass(self, X, fs, fl, fh):
        X = np.swapaxes(X, 1, 2)
        X = bandpass.filter_data(X, fs, fl, fh, verbose=False)
        X = np.swapaxes(X, 1, 2)
        return X
        
    def calcucateCovariances(self, X):
        covlist = np.empty((X.shape[0], X.shape[2], X.shape[2]))
        for i in range(len(X)):
            epoch = X[i]
            covlist[i] = np.dot(epoch.T, epoch)
        return covlist
    
    def separate_classes(self, X, Y):
        classSpecificCOV = []
        for i in range(2): 
            ind = np.where(Y==i)[0]
            classCOV = X[ind]
            classSpecificCOV.append(classCOV)
        return classSpecificCOV
    
    def CSP(self, X, Y, n):
        classSpecificCOV = self.separate_classes(X, Y)
        
        class0_avg = sum(classSpecificCOV[0])/len(classSpecificCOV[0])
        class1_avg = sum(classSpecificCOV[1])/len(classSpecificCOV[1])
        
        _,V = la.eigh(class0_avg, class0_avg+class1_avg)
        V = np.concatenate((V[:, :n], V[:, -n:]), axis=1)
        
        return V
    
    def applyCSP(self, dk, V):
        a = np.dot(np.dot(V.T, dk), V) 
        f = np.log(np.diagonal(a)/np.trace(a)) 
        return f
    
    def get_mi_features(self, x, t, y=None, fs=1000, fl=8, fh=30, n=3):

        channels = [7, 8, 9, 10, 12, 13, 14, 17, 18, 19, 20, 32, 33, 34, 35, 36, 37, 38, 39, 40]
        
        X = np.empty((len(t), 4000, 20))
        
        for i in range(len(t)):
            X[i] = x[t[i]:t[i]+4000, channels]
        
        filteredX = self.bandpass(X, fs, fl, fh)

        covX = self.calcucateCovariances(filteredX)
        if y is not None:
            y = y-1
            V = self.CSP(covX, y, n)
            self.V = V
        else:
            if self.V is None:
                raise Exception("train CSP first please")
            V=self.V
        
        features = np.empty((len(X), 2*n))
  
        for i in range(len(X)):
            epoch = covX[i]
            features[i] = self.applyCSP(epoch, V) 
            
        return features

 DEMO:

In [5]:
def load_data(ses, sub):
    # example: 'data/session1/s1/sess01_subj01_EEG_ERP.mat'
    if len(str(sub))>1:
      filepath = f'data/session{ses}/s{sub}/sess0{ses}_subj{sub}_EEG_MI.mat'
    else:  
      filepath = f'data/session{ses}/s{sub}/sess0{ses}_subj0{sub}_EEG_MI.mat'
      
    train_data = loadmat(filepath)[f'EEG_MI_train'][0][0]
    test_data = loadmat(filepath)[f'EEG_MI_test'][0][0]
    return train_data, test_data

In [6]:
for ses in range(1,3):
  print("Session " + str(ses) + "\n")
  for sub in range(1, 21):
    print("\nSubject " + str(sub) + ":")

    train, test = load_data(ses, sub)
    x_train = train['x']
    t_train = train['t'][0]
    y_train = train['y_dec'][0]

    x_test = test['x']
    t_test = test['t'][0]
    y_test = test['y_dec'][0]

    csp = MI()

    train_features = csp.get_mi_features(x_train, t_train, y_train)
    test_features = csp.get_mi_features(x_test, t_test)

    clf = LDA()
    clf.fit(train_features, y_train)
    acc = clf.score(test_features, y_test)
    print("Accuracy = " + str(acc))
    

Session 1


Subject 1:
Accuracy = 0.78

Subject 2:
Accuracy = 0.92

Subject 3:
Accuracy = 0.92

Subject 4:
Accuracy = 0.49

Subject 5:
Accuracy = 0.86

Subject 6:
Accuracy = 0.78

Subject 7:
Accuracy = 0.51

Subject 8:
Accuracy = 0.54

Subject 9:
Accuracy = 0.82

Subject 10:
Accuracy = 0.63

Subject 11:
Accuracy = 0.48

Subject 12:
Accuracy = 0.5

Subject 13:
Accuracy = 0.52

Subject 14:
Accuracy = 0.55

Subject 15:
Accuracy = 0.56

Subject 16:
Accuracy = 0.53

Subject 17:
Accuracy = 0.72

Subject 18:
Accuracy = 0.87

Subject 19:
Accuracy = 0.74

Subject 20:
Accuracy = 0.53
Session 2


Subject 1:
Accuracy = 0.76

Subject 2:
Accuracy = 0.89

Subject 3:
Accuracy = 0.95

Subject 4:
Accuracy = 0.73

Subject 5:
Accuracy = 0.87

Subject 6:
Accuracy = 0.82

Subject 7:
Accuracy = 0.73

Subject 8:
Accuracy = 0.51

Subject 9:
Accuracy = 0.74

Subject 10:
Accuracy = 0.61

Subject 11:
Accuracy = 0.5

Subject 12:
Accuracy = 0.63

Subject 13:
Accuracy = 0.59

Subject 14:
Accuracy = 0.52

Subject 15:

We need an instance of MI class, since it remembers the spatial filters matrix after being presented with the training data, and, therefore, has a state